In [ ]:
# %load boiler_plate.py
# reload modules when they change
%load_ext autoreload
%autoreload 2

# Alow plotting
%matplotlib inline

# Import modules higher in folder hierarchy
import os
import sys
import logging

from imp import reload

reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

def add_path(path):
    if path not in sys.path:
        sys.path.append(path)

In [ ]:
add_path('..')

In [ ]:
from pipeline.db import fetch, inner_join, filter_videos

# Load data
## Videos data

In [ ]:
dbname='ds-wizards'
user='wizard'
host='192.95.32.117'
password='GaG23jVxZhMnQaU53r8o'

VQUERY = "select post_id, url from videos where status='ok'"

vres = fetch(host, dbname, user, password, VQUERY)
vres = [(post_id.split("_")[1], url) for post_id, url in vres]

## Tags

In [ ]:
dbname='ds-content-tags'
user='ds-content-tags'
password='0fXjWl592vNf1gYvIw8w'
host='192.95.32.117'

TQUERY = "select id, tags from videos where tags is not NULL"
TAGS = "select tag_id, name, path from content_tags"

tres = fetch(host, dbname, user, password, TQUERY)

tags = { 
    tag_id: (name, path) for (tag_id, name, path) in fetch(
        host, dbname, user, password, TAGS) 
}

In [ ]:
import itertools
from collections import Counter

In [ ]:
c = Counter(itertools.chain(*[tags for (_, tags) in tres]))
c_min10 = sorted([(tags[k][0], v  )for k, v in c.items() if v > 10], key = lambda x: x[1], )

In [ ]:
Counter([len(path) for name, path in tags.values()])

In [ ]:
[(name, path) for name, path in tags.values() if len(path) == 2]

## Join videos with tags

In [ ]:
videos = list(inner_join(tres, vres))
filtered, t2i, i2t = filter_videos(videos, 10)
print("Found %d videos with %d unique tags" % (len(filtered), len(t2i)))

# DbPedia

In [ ]:
import requests

from urllib.parse import quote
from xml.etree import ElementTree

In [ ]:
XMLSN = "{http://lookup.dbpedia.org/}"
DBPEDIA_QUERY = "http://lookup.dbpedia.org/api/search/KeywordSearch?&QueryString=%s"
LABEL_PATH = ".//{0}Label".format(XMLSN)
DESCRIPTION_PATH = "{0}Description".format(XMLSN)
RESULT = "{0}Result".format(XMLSN)

In [ ]:
import re

def keep(label):
    if "owl#Thing" in label:
        return False
    
    if "from" in label:
        return False
    
    if "births" in label:
        return False
    
    if "xmlns" in label:
        return False
    
    if re.search(r"american people of [a-z]* descent", label):
        return False
    
    return True

def normalise(label):
    label = label.lower()
    
    return label

In [ ]:
def dbpedia_labels(tag, only_first = True):
    response = requests.get(DBPEDIA_QUERY % quote(tag))
    tree = ElementTree.fromstring(response.content)

    # It there no resutls, return tag and empy list
    if not tree.findall(RESULT):
        raise NameError

    tree = tree[0] if only_first else tree

    # Fetch labels form xml
    labels = [label.text for label in tree.findall(LABEL_PATH)]

    # Clean labels
    labels = [normalise(label) for label in labels if keep(label)]

    # Remove duplicates
    labels = list(set(labels))

    # Get Description
    description = tree.find(DESCRIPTION_PATH).text
    description = description.strip() if description else tag
    
    return description, labels

# Explore DbPedia tags

In [ ]:
for tag_id, (name, path) in itertools.islice(tags.items(), 3):
    try:
        desc, labels =  dbpedia_labels(name)
        print("%s | %s | \n%s" % (tag_id, name, desc))
        for label in labels:
            print("\t %s" % label)
    except NameError:
        print("[Exception] Could not find tags for %s" % name)

In [ ]:
def make_t2l(tags, logging_step=100, only_first = True):
    t2l = {}
    for index, tag_id in enumerate(tags):
        name, path =  tags[tag_id]
        if index % logging_step == 0:
            print("[%d] Fetching info for %s object in dbpedia" % (index, name))
        
        try:
            t2l[tag_id] = dbpedia_labels(name, only_first)
        except NameError:
            print("Could not find label for tag %s" % name)
        except Exception as e:
            print("Exception for %s [%s]" % name, e.message)
        
    return t2l

In [ ]:
t2l = make_t2l(tags)

In [ ]:
labels_raw = itertools.chain(*[set(labels) for (tag_id, (desc, labels)) in t2l.items()])
labels_raw = [normalise(label) for label in labels_raw if keep(label)]
labels = Counter(labels_raw)

In [ ]:
# Get filterd and raw key
keys = set(t2l).intersection(set(tags))

In [ ]:
from collections import namedtuple

# define tag named tuple
Tag = namedtuple('Tag', ['id', 'name', 'description', 'labels'], verbose=True)

# create list of joined tags
joined_tags = {k: Tag(k, tags[k][0], t2l[k][0], t2l[k][1])  for k in keys }

In [ ]:
for tag_id, tag in itertools.islice(joined_tags.items(), 3):
    print("%s | %s | \n%s" % (tag_id, tag.name, tag.description))
    for label in tag.labels:
        print("\t %s" % label)

# IO

In [ ]:
import pickle

## Save tags

In [ ]:
with open("/data/video/dbpedia_tags.pickle", "wb") as handle:
    pickle.dump(t2l, handle)

In [ ]:
with open("/data/video/tag_list.pickle", "wb") as handle:
    pickle.dump(joined_tags, handle)

## Load tags

In [ ]:
with open("/data/video/dbpedia_tags.pickle", "rb") as handle:
    t2l = pickle.load(handle)